# Stworzenie bazy danych do projektu Space-U
**Autorzy:** Karolina Bakalarz, Amelia Bieda, Agnieszka Staszkiewicz, Aleksandra Szczur, Paweł Wojarnik.


Baza danych zawiera kolejno tabele:
1. rodzaje_wypraw
2. kierunek
3. pojazdy
4. wyprawy
5. stanowiska
6. adresy
7. pracownicy
8. klienci
9. kontakt_bliski
10. transakcje
11. metody_platnosci
12. uczestnicy_wyprawy
13. rodzaje_kosztow
14. koszty_organizacji
15. jedzenie
16. alergeny
17. jedzenie_alergen
18. klient_alergen


Importy i konfiguracja połączenia z bazą danych

In [ ]:
import mysql.connector
from faker import Faker
import random
import string
from decimal import Decimal
from datetime import date, timedelta

faker = Faker('pl_PL')

DB_CONFIG = {
    "host": "****.it",
    "user": "****",
    "password": "****",
    "database": "****",
    "port": 0000
}

# rodzaje_wypraw

In [20]:
# Lista wypraw z zakresem cen i dni
wyprawy = [
    ("R01", "Orbitalny weekend", "Krótki lot turystyczny na ISS", (80, 90), (3, 5), "Romantyczna kolacja w gwiazdach", (30, 50), "K01"),
    ("R02", "Misja na księżyc", "Przelot wokół Księżyca bez lądowania", (300, 350), (6, 8), "Degustacja kosmicznych win pod kopułą Księżyca", (20, 30), "K02"),
    ("R03", "Jak podbić księżyc", "Pełna misja z lądowaniem i noclegiem na Księżycu", (550, 600), (10, 14), "Spacer po powierzchni Księżyca", (15, 30), "K02"),
    ("R04", "Pierwszy krok", "Wyprawa na Marsa z 1-miesięcznym pobytem", (11000, 13000), (250, 300), "Symulacja przetrwania na Marsie", (40, 50), "K03"),
    ("R05", "Czerwona stacja", "Wyprawa na Marsa z 3-miesięcznym pobytem", (18000, 22000), (420, 480), "Uprawa roślin w marsjańskiej szklarni", (30, 40), "K03"),
    ("R06", "W poszukiwaniu UFO", "Wyprawa na Marsa z 6-miesięcznym pobytem", (28000, 32000), (580, 620), "Symulacja burzy piaskowej w kontrolowanej komorze środowiskowej", (40, 50), "K03"),
    ("R07", "Gwiezdźisty pas", "Misja badawcza do pasa asteroidów z próbą pobrania materiału", (40000, 45000), (700, 750), "Warsztaty chemiczne", (15, 30), "K04"),
    ("R08", "Oko Ziemi", "Obserwacja Ziemi z orbity polarnej", (100, 120), (2, 3), "Kurs fotografii", (20, 30), "K05"),
    ("R09", "Zimny księżyc", "Lodowa wyprawa na aktywny księżyc Saturna", (70000, 80000), (1100, 1250), "Stworzenie własnego kryształu lodowego z próbki", (40, 60), "K06"),
]

# Połączenie z bazą danych
conn = mysql.connector.connect(**DB_CONFIG)
cursor = conn.cursor()

try:
    # Wyłącz sprawdzanie kluczy obcych dla szybkości
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0")
    cursor.execute("SET autocommit = 0")
    cursor.execute("SET unique_checks = 0")

    # Tworzenie tabeli rodzaje_wypraw
    cursor.execute("DROP TABLE IF EXISTS rodzaje_wypraw")
    cursor.execute("""
    CREATE TABLE rodzaje_wypraw (
        rodzaj_wyprawy_id VARCHAR(10) PRIMARY KEY,
        nazwa VARCHAR(100),
        opis TEXT,
        cena_wyprawy_mln DECIMAL(10, 2),
        czas_trwania_dni INT,
        dodatkowe_aktywnosci TEXT,
        cena_aktywnosci_zl BIGINT,
        kierunek_id VARCHAR(10),
        CONSTRAINT fk_rodzaje_kierunek FOREIGN KEY (kierunek_id)
        REFERENCES kierunek(kierunek_id)
    )
    """)
    
    # Wstawianie danych
    for wyprawa in wyprawy:
        (id_, nazwa, opis, cena_range, dni_range, aktywnosc, aktywnosc_range, kierunek_id) = wyprawa
        cena_wyprawy = round(random.uniform(*cena_range), 2) 
        czas_trwania = random.randint(*dni_range)
        cena_aktywnosci = round(random.uniform(*aktywnosc_range), 2) * 1000
    
        cursor.execute("""
            INSERT INTO rodzaje_wypraw (
                rodzaj_wyprawy_id, nazwa, opis, cena_wyprawy_mln, czas_trwania_dni,
                dodatkowe_aktywnosci, cena_aktywnosci_zl, kierunek_id
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            id_, nazwa, opis, cena_wyprawy, czas_trwania,
            aktywnosc, cena_aktywnosci, kierunek_id
        ))

    # Przywróć ustawienia bazy danych
    cursor.execute("SET FOREIGN_KEY_CHECKS = 1")
    cursor.execute("SET autocommit = 1")
    cursor.execute("SET unique_checks = 1")
    conn.commit()
        
except mysql.connector.Error as err:
    print(f"Błąd: {err}")
    conn.rollback()

finally:
    if conn.is_connected():
        cursor.close()
        conn.close()

# kierunek

In [4]:
# Dane kierunków
kierunki = [
    ("K01", "ISS"),
    ("K02", "Księżyc"),
    ("K03", "Mars"),
    ("K04", "Pas asteroidów"),
    ("K05", "Orbita polarna Ziemi"),
    ("K06", "Enceladus")
]

# Połączenie z bazą danych
conn = mysql.connector.connect(**DB_CONFIG)
cursor = conn.cursor()

try:
    # Wyłącz sprawdzanie kluczy obcych dla szybkości
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0")
    cursor.execute("SET autocommit = 0")
    cursor.execute("SET unique_checks = 0")
    
    # Tworzenie tabeli kierunek
    cursor.execute("DROP TABLE IF EXISTS kierunek")
    cursor.execute("""
    CREATE TABLE kierunek (
        kierunek_id VARCHAR(10) PRIMARY KEY,
        nazwa VARCHAR(50) NOT NULL
    )
    """)
    
    # Wstawianie danych do tabeli kierunek
    for kierunek in kierunki:
        cursor.execute("INSERT INTO kierunek (kierunek_id, nazwa) VALUES (%s, %s)", kierunek)

    # Przywróć ustawienia bazy danych
    cursor.execute("SET FOREIGN_KEY_CHECKS = 1")
    cursor.execute("SET autocommit = 1")
    cursor.execute("SET unique_checks = 1")
    conn.commit()

except Exception as e:
    conn.rollback()
    print(f"Błąd: {e}")
finally:
    cursor.close()
    conn.close()

# pojazdy

In [5]:
def generuj_nazwe_pojazdu():
    """Funkcja do generowania nazw pojazdów"""
    znaki_brzeg = string.ascii_uppercase + string.digits
    znaki_srodek = string.ascii_uppercase + string.digits + 10*"-"
    dlugosc = random.randint(5, 10)

    # Pierwszy i ostatni znak bez myślnika
    pierwszy = random.choice(znaki_brzeg)
    ostatni = random.choice(znaki_brzeg)

    # Generuj środek bez podwójnych myślników
    srodek = []
    poprzedni = ""
    for _ in range(dlugosc - 2):
        znaki = znaki_srodek if poprzedni != "-" else string.ascii_uppercase + string.digits
        znak = random.choice(znaki)
        srodek.append(znak)
        poprzedni = znak

    return pierwszy + ''.join(srodek) + ostatni


def losuj_date_produkcji():
    """Funkcja do generowania daty produkcji z lat 2018–2021"""
    rok = random.randint(2018, 2021)
    miesiac = random.randint(1, 12)
    dzien = random.randint(1, 28)  # uproszczenie
    return date(rok, miesiac, dzien)


def losuj_date_kontroli():
    """Funkcja do generowania daty kontroli z roku 2023 lub 2024"""
    rok = random.choice([2024, 2025])
    
    if rok == 2024:
        miesiac = random.randint(1, 12)
        dzien = random.randint(1, 28)
        return date(rok, miesiac, dzien)
    
    else:
        # Rok 2025 – tylko daty do dziś
        today = date.today()
        # Losowy dzień z zakresu od 1 stycznia do dziś
        start_2025 = date(2025, 1, 1)
        zakres_dni = (today - start_2025).days
        if zakres_dni < 0:
            # Jeśli dzisiejsza data to np. grudzień 2024 – nie losujemy nic z 2025
            return losuj_date_kontroli()  # spróbuj ponownie (wylosuje 2024)
        losowa_data = start_2025 + timedelta(days=random.randint(0, zakres_dni))
        return losowa_data

# Połączenie z bazą
conn = mysql.connector.connect(**DB_CONFIG)
cursor = conn.cursor()
try:
    # Tworzenie tabeli
    cursor.execute("DROP TABLE IF EXISTS pojazdy")
    cursor.execute("""
    CREATE TABLE pojazdy (
        pojazd_id VARCHAR(10) PRIMARY KEY,
        nazwa VARCHAR(50),
        typ VARCHAR(50),
        maksymalny_udzwig_w_tonach INT,
        liczba_miejsc INT,
        data_produkcji DATE,
        data_ostatniej_kontroli DATE,
        producent VARCHAR(50)
    )
    """)
    
    # Generowanie i wstawianie pojazdów
    typy = ["wycieczkowy", "wycieczkowy", "wycieczkowy", "wycieczkowy", "treningowy"]
    random.shuffle(typy)
    
    producenci = ["Supernova", "Blue_sky", "New_planet", "Galaxy_way"]
    
    for i, typ in enumerate(typy, start=1):
        pojazd_id = f"Pojazd_{i}"
        nazwa = generuj_nazwe_pojazdu()
    
        if typ == "treningowy":
            udzwig = random.randint(80, 90)
            miejsca = 4
        else:
            udzwig = random.randint(90, 120)
            miejsca = random.randint(6, 8)
    
        data_produkcji = losuj_date_produkcji()
        data_kontroli = losuj_date_kontroli()
        producent = random.choice(producenci)
    
        cursor.execute("""
            INSERT INTO pojazdy (
                pojazd_id, nazwa, typ, maksymalny_udzwig_w_tonach,
                liczba_miejsc, data_produkcji, data_ostatniej_kontroli, producent
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            pojazd_id, nazwa, typ, udzwig, miejsca,
            data_produkcji, data_kontroli, producent
        ))

    conn.commit()

except Exception as e:
    conn.rollback()
    print(f"Błąd: {e}")
finally:
    cursor.close()
    conn.close()

# wyprawy

In [6]:
# Połączenie z bazą danych
conn = mysql.connector.connect(**DB_CONFIG)
cursor = conn.cursor(dictionary=True)

try:
    # Wyłącz sprawdzanie kluczy obcych dla szybkości
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0")
    cursor.execute("SET autocommit = 0")
    cursor.execute("SET unique_checks = 0")

    # 1. Pobieranie danych w jednym zapytaniu (zamiast dwóch)
    cursor.execute("""
        SELECT p.pojazd_id, p.data_produkcji, 
               r.rodzaj_wyprawy_id, r.czas_trwania_dni 
        FROM pojazdy p
        CROSS JOIN rodzaje_wypraw r
        WHERE p.typ = 'wycieczkowy'
    """)
    
    # Przetwarzanie danych do bardziej wydajnych struktur
    pojazdy = {}
    rodzaje_wypraw = []
    
    for row in cursor:
        if row['pojazd_id'] not in pojazdy:
            pojazdy[row['pojazd_id']] = {
                'data_produkcji': row['data_produkcji'],
                'zajete_okresy': []
            }
        rodzaje_wypraw.append((row['rodzaj_wyprawy_id'], row['czas_trwania_dni']))

    # 2. Tworzenie tabeli z użyciem buforowania
    cursor.execute("DROP TABLE IF EXISTS wyprawy")
    cursor.execute("""
        CREATE TABLE wyprawy (
            wyprawa_id VARCHAR(10) PRIMARY KEY,
            data_startu DATE,
            data_powrotu DATE,
            pojazd_id VARCHAR(10),
            rodzaj_wyprawy_id VARCHAR(10),
            status VARCHAR(20),
            INDEX (pojazd_id),
            INDEX (data_startu),
            INDEX (data_powrotu)
            )
    """)

    # 3. Generowanie wypraw z użyciem bardziej wydajnych struktur danych
    today = date.today()
    wyprawy = []
    pojazdy_list = list(pojazdy.items())
    
    # Generowanie 20 unikalnych wypraw
    for i in range(1, 21):
        wyprawa_id = f"W{i:02d}"
        rodzaj_id, czas_trwania = random.choice(rodzaje_wypraw)
        
        # Wybierz pojazd z najmniejszą liczbą zaplanowanych wypraw (lepsze rozłożenie)
        pojazd_id, pojazd_data = min(pojazdy_list, key=lambda x: len(x[1]['zajete_okresy']))
        
        earliest_date = pojazd_data['data_produkcji']
        max_start_date = earliest_date + timedelta(days=2000)
        
        # Generuj daty bez kolizji
        for _ in range(100):  # Limit prób
            proposed_start = earliest_date + timedelta(days=random.randint(0, (max_start_date - earliest_date).days))
            proposed_end = proposed_start + timedelta(days=czas_trwania)
            
            # Sprawdzenie kolizji z optymalizacją
            if not any(not (proposed_end < start or proposed_start > end) 
                      for start, end in pojazd_data['zajete_okresy']):
                pojazd_data['zajete_okresy'].append((proposed_start, proposed_end))
                status = "zakończona" if proposed_end < today else "zaplanowana"
                wyprawy.append((wyprawa_id, proposed_start, proposed_end, pojazd_id, rodzaj_id, status))
                break

    # 4. Masowe wstawianie danych z użyciem executemany
    if wyprawy:
        cursor.executemany("""
            INSERT INTO wyprawy (
                wyprawa_id, data_startu, data_powrotu,
                pojazd_id, rodzaj_wyprawy_id, status
            ) VALUES (%s, %s, %s, %s, %s, %s)
        """, wyprawy)

    # Przywróć ustawienia bazy danych
    cursor.execute("SET FOREIGN_KEY_CHECKS = 1")
    cursor.execute("SET autocommit = 1")
    cursor.execute("SET unique_checks = 1")
    conn.commit()

except Exception as e:
    conn.rollback()
    print(f"Błąd: {e}")
finally:
    cursor.close()
    conn.close()

# stanowiska

In [ ]:
conn = mysql.connector.connect(**DB_CONFIG)
cursor = conn.cursor()

try:
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0")
    cursor.execute("DROP TABLE IF EXISTS stanowiska")
    
    cursor.execute("""
        CREATE TABLE stanowiska (
            stanowisko_id INT PRIMARY KEY AUTO_INCREMENT,
            nazwa VARCHAR(100) UNIQUE
        )
    """)
    cursor.execute("ALTER TABLE stanowiska AUTO_INCREMENT = 1")
    stanowiska_lista = [
        'Pilot statku kosmicznego',
        'Inżynier pokładowy',
        'Technik rakietowy',
        'Kontroler lotów',
        'Lekarz misji',
        'Specjalista ds. bezpieczeństwa',
        'Pracownik biura podróży',
        'Księgowy',
        'PR Manager',
        'Właściciel firmy'
    ]
    stanowiska_lista.sort()
    

    for nazwa in stanowiska_lista:
        cursor.execute("INSERT INTO stanowiska (nazwa) VALUES (%s)", (nazwa,))

    cursor.execute("SELECT stanowisko_id, nazwa FROM stanowiska ORDER BY stanowisko_id")
    stanowiska_w_bazie = cursor.fetchall()

    cursor.execute("SET FOREIGN_KEY_CHECKS = 1")
    conn.commit()

except Exception as e:
    conn.rollback()
    print(f"Błąd: {e}")
finally:
    cursor.close()
    conn.close()

# adresy

In [8]:
# Połączenie z bazą danych
conn = mysql.connector.connect(**DB_CONFIG)
cursor = conn.cursor()

try:
    # Wyłącz sprawdzanie kluczy obcych 
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0")

    # Stworzenie tabeli adresy
    cursor.execute("DROP TABLE IF EXISTS adresy")
    cursor.execute("""
    CREATE TABLE adresy (
        adres_id INT AUTO_INCREMENT PRIMARY KEY,
        miasto VARCHAR(50),
        ulica_numer VARCHAR(100),
        kod_pocztowy VARCHAR(10)
    )
    """)
    
    # Przywróć ustawienia bazy danych
    cursor.execute("SET FOREIGN_KEY_CHECKS = 1")
    conn.commit()

except Exception as e:
    conn.rollback()
    print(f"Błąd: {e}")
finally:
    cursor.close()
    conn.close()

# pracownicy

In [ ]:
conn = mysql.connector.connect(**DB_CONFIG)
cursor = conn.cursor()

try:
    cursor.execute("DROP TABLE IF EXISTS pracownicy")

    cursor.execute("""
        CREATE TABLE pracownicy (
            pracownik_id VARCHAR(10) PRIMARY KEY,
            plec VARCHAR(1),
            imie VARCHAR(50),
            nazwisko VARCHAR(50),
            data_urodzenia DATE,
            adres_id INT,
            stanowisko_id INT,
            staz_w_firmie VARCHAR(20),
            wynagrodzenie_miesięczne DECIMAL(10,2),
            email VARCHAR(100),
            telefon VARCHAR(20),
            CONSTRAINT fk_pracownicy_stanowiska FOREIGN KEY (stanowisko_id) REFERENCES stanowiska(stanowisko_id),
            CONSTRAINT fk_pracownicy_adresy FOREIGN KEY (adres_id) REFERENCES adresy(adres_id)
        )
    """)

    cursor.execute("SELECT stanowisko_id, nazwa FROM stanowiska")
    stanowiska_w_bazie = cursor.fetchall()

    pilot_id = next((id for id, nazwa in stanowiska_w_bazie if nazwa == "Pilot statku kosmicznego"), None)
    if pilot_id is None:
        raise ValueError("Nie znaleziono stanowiska 'Pilot statku kosmicznego' w tabeli stanowiska.")

    # Określenie liczby pracowników
    liczba_pracownikow = len(stanowiska_w_bazie) + 5  

    # Przygotowanie listy losowych stanowisk z gwarancją 3 pilotów
    losowe_stanowiska = [pilot_id] * 3
    wszystkie_id = [id for id, _ in stanowiska_w_bazie]
    losowe_stanowiska += [random.choice(wszystkie_id) for _ in range(liczba_pracownikow - 3)]
    random.shuffle(losowe_stanowiska)

    miasta_wroclaw = [
        "Wrocław", "Oleśnica", "Oława", "Trzebnica", "Strzelin",
        "Brzeg Dolny", "Oborniki Śląskie", "Kąty Wrocławskie", "Siechnice",
        "Długołęka", "Środa Śląska", "Czernica", "Żórawina", "Miękinia", "Tyniec Mały"
    ]

    for i, stanowisko_id in enumerate(losowe_stanowiska, 1):
        pracownik_id = f"P_{i+1:04d}"
        plec = random.choice(["M", "K"])
        imie = faker.first_name_male() if plec == "M" else faker.first_name_female()
        nazwisko = faker.last_name()
        imie_clean = imie.lower().translate(str.maketrans("ąćęłńóśżź", "acelnoszz"))
        nazwisko_clean = nazwisko.lower().translate(str.maketrans("ąćęłńóśżź", "acelnoszz"))
        email = f"{imie_clean}.{nazwisko_clean}@spaceu.com"
        telefon = f"{random.randint(100, 999)} {random.randint(100, 999)} {random.randint(100, 999)}"
        data_urodzenia = faker.date_of_birth(minimum_age=25, maximum_age=60 if plec == "K" else 65)

        stanowisko = next(nazwa for id, nazwa in stanowiska_w_bazie if id == stanowisko_id)

        if stanowisko == "Pilot statku kosmicznego":
            wynagrodzenie = round(random.uniform(30000, 35000), 2)
        elif stanowisko == "Właściciel firmy":
            wynagrodzenie = round(random.uniform(45000, 50000), 2)
        elif stanowisko == "Lekarz misji":
            wynagrodzenie = round(random.uniform(20000, 25000), 2)
        elif stanowisko == "Inżynier pokładowy":
            wynagrodzenie = round(random.uniform(15000, 18000), 2)
        elif stanowisko in ["Technik rakietowy", "Specjalista ds. bezpieczeństwa"]:
            wynagrodzenie = round(random.uniform(10000, 15000), 2)
        elif stanowisko == "Pracownik biura podróży":
            wynagrodzenie = round(random.uniform(6000, 7000), 2)
        elif stanowisko == "Księgowy":
            wynagrodzenie = round(random.uniform(7000, 8000), 2)
        elif stanowisko == "PR Manager":
            wynagrodzenie = round(random.uniform(8000, 9000), 2)
        else:
            wynagrodzenie = round(random.uniform(10000, 15000), 2)

        staz_w_firmie = ">5 lat" if stanowisko == "Właściciel firmy" else random.choice(["<1 rok", "1-2 lata", "2-3 lata", "3-4 lata", ">5 lat"])

        miasto = random.choice(miasta_wroclaw)
        ulica = faker.street_name()
        numer_budynku = faker.building_number()
        ulica_numer = f"{ulica} {numer_budynku}"
        kod_pocztowy = faker.postcode()

        cursor.execute("""
            INSERT INTO adresy (miasto, ulica_numer, kod_pocztowy) 
            VALUES (%s, %s, %s)
        """, (miasto, ulica_numer, kod_pocztowy))
        adres_id = cursor.lastrowid

        cursor.execute("""
            INSERT INTO pracownicy (
                pracownik_id, plec, imie, nazwisko, data_urodzenia, 
                adres_id, stanowisko_id, staz_w_firmie, 
                wynagrodzenie_miesięczne, email, telefon
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            pracownik_id, plec, imie, nazwisko, data_urodzenia,
            adres_id, stanowisko_id, staz_w_firmie,
            wynagrodzenie, email, telefon
        ))

    conn.commit()

except Exception as e:
    conn.rollback()
    print(f"Błąd: {e}")
finally:
    cursor.close()
    conn.close()


# klienci

In [10]:
# Połączenie z bazą danych
conn = mysql.connector.connect(**DB_CONFIG)
cursor = conn.cursor()

try:
    # Wyłącz sprawdzanie kluczy obcych 
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0")
    cursor.execute("DROP TABLE IF EXISTS klienci")
    cursor.execute("""
    CREATE TABLE klienci (
        klient_id INT AUTO_INCREMENT PRIMARY KEY,
        plec VARCHAR(1),
        imie VARCHAR(50),
        nazwisko VARCHAR(50),
        e_mail VARCHAR(100),
        telefon VARCHAR(20),
        data_urodzenia DATE,
        adres_id INT,
        czy_wegetarian BOOLEAN,
        czy_weganin BOOLEAN,
        alergeny TEXT,
        waga DECIMAL(5,2),
        wzrost INT,
        FOREIGN KEY (adres_id) REFERENCES adresy(adres_id)
    )
    """)
    
    # Generowanie danych dla 30 klientów
    adresy_wygenerowane = []
    for _ in range(30):
        miasto = faker.city()
        ulica_numer = f"{faker.street_name()} {faker.building_number()}"
        kod_pocztowy = faker.postcode()
        cursor.execute("INSERT INTO adresy (miasto, ulica_numer, kod_pocztowy) VALUES (%s, %s, %s)",
                       (miasto, ulica_numer, kod_pocztowy))
        adresy_wygenerowane.append(cursor.lastrowid)  
        
        
    for adres_id in adresy_wygenerowane:
        plec = random.choice(["M", "K"])
        imie = faker.first_name_male() if plec == "M" else faker.first_name_female()
        nazwisko = faker.last_name()
        imie_clean = imie.lower().translate(str.maketrans("ąćęłńóśżź","acelnoszz"))
        nazwisko_clean = nazwisko.lower().translate(str.maketrans("ąćęłńóśżź","acelnoszz"))
        e_mail = f"{imie_clean}.{nazwisko_clean}@example.com"
        numer = ''.join(str(random.randint(0, 9)) for _ in range(9))
        telefon = f"+48 {numer[:3]} {numer[3:6]} {numer[6:]}"
        data_urodzenia = faker.date_of_birth(minimum_age=18, maximum_age=70)
        czy_weganin = random.choice([True, False])
        czy_wegetarian = czy_weganin or random.choice([True, False])
        alergeny_list = random.sample(
            ["gluten", "orzechy", "laktoza", "jaja", "ryby", "soja", "seler"],
            k=random.randint(0, 3)
        )
        alergeny = ",".join(alergeny_list)
        waga = round(random.uniform(50, 110), 2)
        wzrost = random.randint(150, 200)
    
        cursor.execute("""
            INSERT INTO klienci (
                plec, imie, nazwisko, e_mail, telefon, data_urodzenia, adres_id,
                czy_wegetarian, czy_weganin, alergeny, waga, wzrost
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            plec, imie, nazwisko, e_mail, telefon, data_urodzenia, adres_id,
            czy_wegetarian, czy_weganin, alergeny, waga, wzrost
        ))
        
    # Przywróć ustawienia bazy danych
    cursor.execute("SET FOREIGN_KEY_CHECKS = 1")
    conn.commit()

except Exception as e:
    conn.rollback()
    print(f"Błąd: {e}")
finally:
    cursor.close()
    conn.close()

# kontakt_bliski

In [19]:
# Połączenie z bazą danych
conn = mysql.connector.connect(**DB_CONFIG)
cursor = conn.cursor(dictionary=True)

try:
    # Tworzenie tabeli kontakt_bliski
    cursor.execute("DROP TABLE IF EXISTS kontakt_bliski")
    cursor.execute("""
    CREATE TABLE kontakt_bliski (
        b_klienta_id VARCHAR(20) PRIMARY KEY,
        klient_id VARCHAR(10),
        imie VARCHAR(50),
        nazwisko VARCHAR(50),
        relacja VARCHAR(20),
        telefon VARCHAR(20),
        email VARCHAR(100)
    )
    """)
    
    # Pobranie klientów
    cursor.execute("SELECT klient_id, nazwisko, data_urodzenia FROM klienci")
    lista_klientow = cursor.fetchall()
    
    # Dane pomocnicze
    relacje_dorosly = ["żona", "mąż", "siostra", "brat", "matka", "ojciec", "znajomy"]
    relacje_mlody = ["siostra", "brat", "matka", "ojciec", "znajomy"]
    b_id_start = "BK"
    bliscy = []
    
    # Generowanie danych osób bliskich
    for i, klient in enumerate(lista_klientow):
        klient_id = klient["klient_id"]
        
        b_klienta_id = f"BK_{klient_id+1:04d}"
        nazwisko = klient["nazwisko"]
        data_urodzenia = klient["data_urodzenia"]
        wiek = date.today().year - data_urodzenia.year
    
        # Dobór relacji
        if wiek < 25:
            relacja = random.choice(relacje_mlody)
        else:
            relacja = random.choice(relacje_dorosly)
    
        # Dane osoby bliskiej
        plec_bliskiej = "K" if relacja in ["żona", "siostra", "matka"] else "M"
        imie_bliskiego = faker.first_name_female() if plec_bliskiej == "K" else faker.first_name_male()
        nazwisko_bliskiego = nazwisko if relacja in ["żona", "mąż"] else faker.last_name()
    
        numer = ''.join([str(random.randint(0, 9)) for _ in range(9)])
        telefon_bliskiego = f"+48 {numer[:3]} {numer[3:6]} {numer[6:]}"
        imie_clean = imie_bliskiego.lower().translate(str.maketrans("ąćęłńóśżź","acelnoszz"))
        nazwisko_clean = nazwisko_bliskiego.lower().translate(str.maketrans("ąćęłńóśżź","acelnoszz"))
        email_bliskiego = f"{imie_clean}.{nazwisko_clean}@gmail.com"
    
        cursor.execute("""
            INSERT INTO kontakt_bliski (
                b_klienta_id, klient_id, imie, nazwisko, relacja, telefon, email
            ) VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (
            b_klienta_id,
            klient_id,
            imie_bliskiego,
            nazwisko_bliskiego,
            relacja,
            telefon_bliskiego,
            email_bliskiego
        ))
    conn.commit()

except Exception as e:
    conn.rollback()
    print(f"Błąd: {e}")
finally:
    cursor.close()
    conn.close()

# transakcje, metody platnosci

In [12]:
# Połączenie z bazą danych
conn = mysql.connector.connect(**DB_CONFIG)
cursor = conn.cursor(dictionary=True)

try: 
    cursor.execute("SELECT wyprawa_id, rodzaj_wyprawy_id, data_startu FROM wyprawy")
    
    wyprawy = cursor.fetchall()
    
    cursor.execute("SELECT klient_id FROM klienci")
    klienci = cursor.fetchall()
    
    cursor.execute("SELECT rodzaj_wyprawy_id, cena_wyprawy_mln, cena_aktywnosci_zl FROM rodzaje_wypraw")
    rodzaje_dict = {
        row["rodzaj_wyprawy_id"]: {
            "cena_wyprawy_mln": float(row["cena_wyprawy_mln"]),
            "cena_aktywnosci_zl": int(row["cena_aktywnosci_zl"])
        }
        for row in cursor.fetchall()
    }
    
    cursor.execute("DROP TABLE IF EXISTS transakcje")
    cursor.execute("""
        CREATE TABLE transakcje (
            transakcja_id VARCHAR(10) PRIMARY KEY,
            klient_id INT,
            wyprawa_id VARCHAR(10),
            data_transakcji DATE,
            kwota_za_wyprawe_mln DECIMAL(10, 2),
            czy_dod_atrakcja VARCHAR(5),
            kwota_atrakcji INT,
            metoda_platnosci VARCHAR(30)
        )
    """)
    
    metody = ["karta", "karta", "karta", "karta", "karta", "bitcoin"]
    
    for i, klient in enumerate(klienci):
        transakcja_id = f"T{str(i+1).zfill(4)}"
        klient_id = klient["klient_id"]
        
        wyp = random.choice(wyprawy)  # losowa wyprawa
        wyprawa_id = wyp["wyprawa_id"]
        rodzaj_id = wyp["rodzaj_wyprawy_id"]
        
        data_transakcji = wyp["data_startu"] - timedelta(days=random.randint(30, 60))
        
        if data_transakcji > date.today():
            data_transakcji = None
    
        kwota_wyprawy = rodzaje_dict[rodzaj_id]["cena_wyprawy_mln"]
        dod_atrakcja = random.choice(["Tak", "Nie"])
        kwota_atrakcji = rodzaje_dict[rodzaj_id]["cena_aktywnosci_zl"] if dod_atrakcja == "Tak" else 0
        metoda = random.choice(metody)
    
        cursor.execute("""
            INSERT INTO transakcje (
                transakcja_id, klient_id, wyprawa_id, data_transakcji,
                kwota_za_wyprawe_mln, czy_dod_atrakcja, kwota_atrakcji, metoda_platnosci
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            transakcja_id, klient_id, wyprawa_id, data_transakcji,
            kwota_wyprawy, dod_atrakcja, kwota_atrakcji, metoda
        ))
        
    # Nowa tabela - metody platnosci
    cursor.execute("""
        DROP TABLE IF EXISTS metody_platnosci
    """)
    cursor.execute("""
        CREATE TABLE metody_platnosci (
        metoda_id INT PRIMARY KEY AUTO_INCREMENT,
        nazwa VARCHAR(50) UNIQUE
    )
    """)
    
    cursor.execute("""
        INSERT INTO metody_platnosci (nazwa) VALUES
        ('karta'),
        ('bitcoin')
    """)
    cursor.execute("""    
        ALTER TABLE transakcje ADD metoda_id INT
    """)
    cursor.execute("""
        UPDATE transakcje t
        JOIN metody_platnosci mp ON t.metoda_platnosci = mp.nazwa
        SET t.metoda_id = mp.metoda_id
    """)
    cursor.execute("""
        ALTER TABLE transakcje DROP COLUMN metoda_platnosci
    """)
    cursor.execute("""
        ALTER TABLE transakcje
        ADD CONSTRAINT fk_transakcje_metoda
        FOREIGN KEY (metoda_id) REFERENCES metody_platnosci(metoda_id)
    """)
    conn.commit()

except Exception as e:
    conn.rollback()
    print(f"Błąd: {e}")
finally:
    cursor.close()
    conn.close()

# uczestnicy_wypraw

In [13]:
try:
    # Nawiązanie połączenia
    conn = mysql.connector.connect(**DB_CONFIG)
    cursor = conn.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS uczestnicy_wyprawy (
        klient_id INT,
        wyprawa_id VARCHAR(10),
        PRIMARY KEY (klient_id, wyprawa_id),
        FOREIGN KEY (klient_id) REFERENCES klienci(klient_id),
        FOREIGN KEY (wyprawa_id) REFERENCES wyprawy(wyprawa_id)
    );
    """)
    
    #  liczby miejsc dla wypraw przez pojazd
    cursor.execute("""
        SELECT w.wyprawa_id, w.pojazd_id, p.liczba_miejsc
        FROM wyprawy w
        JOIN pojazdy p ON w.pojazd_id = p.pojazd_id
    """)
    wyprawy = cursor.fetchall()  
    
    # klienci przypisani w transakcjach
    cursor.execute("SELECT klient_id, wyprawa_id FROM transakcje")
    transakcje = cursor.fetchall()
    
    # transakcje wg wyprawy
    from collections import defaultdict
    mapa_wypraw = defaultdict(list)
    for klient_id, wyprawa_id in transakcje:
        mapa_wypraw[wyprawa_id].append(klient_id)
    
    # tylko tylu uczestników, ile jest miejsc
    for wyprawa_id, pojazd_id, liczba_miejsc in wyprawy:
        uczestnicy = mapa_wypraw.get(wyprawa_id, [])
        uczestnicy = uczestnicy[:liczba_miejsc]  # ogranicz do liczby miejsc
    
        for klient_id in uczestnicy:
            cursor.execute("""
                INSERT IGNORE INTO uczestnicy_wyprawy (klient_id, wyprawa_id)
                VALUES (%s, %s);
            """, (klient_id, wyprawa_id))
    
    conn.commit()

except mysql.connector.Error as err:
    print(f"Błąd: {err}")
    conn.rollback()

finally:
    if conn.is_connected():
        cursor.close()
        conn.close()

# koszty_organizacji, rodzaj_kosztu_organizacyjnego

In [14]:
try:
    # Nawiązanie połączenia
    conn = mysql.connector.connect(**DB_CONFIG)
    cursor = conn.cursor()
    # Optymalizacja - wyłączenie sprawdzania kluczy obcych
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0")
    cursor.execute("SET autocommit = 0")
    cursor.execute("SET unique_checks = 0")
    
    # Tworzenie tabeli z rodzajami kosztów
    cursor.execute("DROP TABLE IF EXISTS rodzaje_kosztow")
    cursor.execute("""
        CREATE TABLE rodzaje_kosztow (
            rodzaj_kosztu_id INT PRIMARY KEY AUTO_INCREMENT,
            nazwa VARCHAR(100),
            min_kwota DECIMAL(15,2),
            max_kwota DECIMAL(15,2)
        )
    """)
    
    # Poprawiona struktura danych - płaska lista krotek
    kategorie_kosztow = [
        ('Paliwo', Decimal('1000000.00'), Decimal('25000000.00')),
        ('Pozwolenie na wejście w atmosferę', Decimal('100000.00'), Decimal('500000.00')),
        ('Jedzenie i picie', Decimal('50000.00'), Decimal('200000.00')),
        ('Tlen', Decimal('20000.00'), Decimal('100000.00'))
    ]
    
    # Wstawianie rodzajów kosztów
    cursor.executemany("""
        INSERT INTO rodzaje_kosztow (nazwa, min_kwota, max_kwota)
        VALUES (%s, %s, %s)
    """, kategorie_kosztow)
    
    # Tworzenie tabeli z kosztami
    cursor.execute("DROP TABLE IF EXISTS koszty_organizacji")
    cursor.execute("""
        CREATE TABLE koszty_organizacji (
            koszt_id INT PRIMARY KEY AUTO_INCREMENT,
            wyprawa_id VARCHAR(10),
            rodzaj_kosztu_id INT,
            kwota_pln DECIMAL(15,2),
            FOREIGN KEY (rodzaj_kosztu_id) REFERENCES rodzaje_kosztow(rodzaj_kosztu_id)
        )
    """)

    def generate_koszty():
        # Pobierz wszystkie wyprawy
        cursor.execute("SELECT wyprawa_id FROM wyprawy")
        wyprawy = [row[0] for row in cursor.fetchall()]
    
        # Pobierz wszystkie rodzaje kosztów
        cursor.execute("SELECT rodzaj_kosztu_id, min_kwota, max_kwota FROM rodzaje_kosztow")
        koszty_rodzaje = cursor.fetchall()
    
        licznik = 0
        for wyprawa_id in wyprawy:
            for rodzaj_id, min_kwota, max_kwota in koszty_rodzaje:
                kwota = Decimal(str(round(random.uniform(float(min_kwota), float(max_kwota)), 2)))
                cursor.execute("""
                    INSERT INTO koszty_organizacji (wyprawa_id, rodzaj_kosztu_id, kwota_pln)
                    VALUES (%s, %s, %s)
                """, (wyprawa_id, rodzaj_id, kwota))
                licznik += 1
    
        conn.commit()
        print(f"Wygenerowano {licznik} rekordów do tabeli koszty_organizacji.")

    generate_koszty()

except mysql.connector.Error as err:
    print(f"Błąd: {err}")
    conn.rollback()

finally:
    if conn.is_connected():
        cursor.close()
        conn.close()

Wygenerowano 64 rekordów do tabeli koszty_organizacji.


# jedzenie

In [15]:
# Dane do wstawienia
jedzenie_data = [
    # Przekąski (J01-J14)
    ('J01', 'batonik czekoladowy', 250, 'przekąska', True, False, '', 'mleko, orzechy, soja, gluten'),
    ('J02', 'chipsy', 520, 'przekąska', True, True, '', 'gluten'),
    ('J03', 'wafle ryżowe', 90, 'przekąska', True, True, '', 'gluten'),
    ('J04', 'żelki', 350, 'przekąska', True, False, '', 'żelatyna'),
    ('J05', 'czekolada', 500, 'przekąska', True, False, '', 'mleko, soja, orzechy'),
    ('J06', 'batonik proteinowy', 280, 'przekąska', True, False, '', 'mleko, soja, orzechy'),
    ('J07', 'tiramisu', 360, 'przekąska', True, False, '', 'mleko, jaja, gluten'),
    ('J08', 'krakersy', 450, 'przekąska', True, True, 'Słone ciasteczka, z dodatkiem ziół lub sera.', 'gluten, sezam'),
    ('J09', 'orzeszki', 500, 'przekąska', True, True, 'Solone lub prażone orzechy ziemne.', 'orzechy ziemne'),
    ('J10', 'paluszki', 380, 'przekąska', True, True, 'Cienkie, chrupiące pieczywo solone.', 'gluten, sezam'),
    ('J11', 'popcorn', 150, 'przekąska', True, True, 'Prażona kukurydza, podawana z solą lub masłem.', 'brak'),
    ('J12', 'ciastka pieguski', 210, 'przekąska', True, False, 'Ciastka z kawałkami czekolady.', 'gluten, mleko, jaja, soja'),
    ('J13', 'skyr pitny', 200, 'przekąska', True, False, 'Gęsty jogurt naturalny typu islandzkiego w formie do picia.', 'mleko'),
    ('J14', 'drożdżówka z serem', 330, 'przekąska', True, False, 'Słodka bułka z nadzieniem twarogowym i kruszonką.', 'gluten, mleko, jaja, soja'),
    
    # Śniadania (J15-J29)
    ('J15', 'owsianka z cynamonem i jabłkiem', 250, 'śniadanie', True, True, 'Ciepła owsianka z duszonym jabłkiem i przyprawami.', 'gluten'),
    ('J16', 'omlet z warzywami i serem', 300, 'śniadanie', True, False, 'Jajeczny placek z dodatkiem warzyw i sera.', 'jaja, mleko'),
    ('J17', 'jajecznica ze szczypiorkiem', 200, 'śniadanie', True, False, 'Klasyczna jajecznica z posiekanym szczypiorkiem.', 'jaja'),
    ('J18', 'jajecznica z boczkiem', 300, 'śniadanie', False, False, 'Jajecznica z dodatkiem podsmażonego boczku.', 'jaja'),
    ('J19', 'pancakes z kremem orzechowo-czekoladowym', 400, 'śniadanie', True, False, 'Amerykańskie naleśniki z kremem typu Nutella.', 'gluten, jaja, mleko, orzechy'),
    ('J20', 'kanapka z szynką, serem i pomidorem', 350, 'śniadanie', False, False, 'Klasyczna kanapka na pieczywie z dodatkami.', 'gluten, mleko'),
    ('J21', 'kanapka z pastą jajeczną', 300, 'śniadanie', True, False, 'Pieczywo z pastą z jajek, majonezu i przypraw.', 'gluten, jaja'),
    ('J22', 'kanapka z mozarellą', 320, 'śniadanie', True, False, 'Kanapka z mozzarellą, pomidorem i bazylią.', 'gluten, mleko'),
    ('J23', 'jogurt naturalny z granolą i owocami', 300, 'śniadanie', True, False, 'Lekki posiłek z chrupiącą granolą i świeżymi owocami.', 'mleko, gluten, orzechy'),
    ('J24', 'serek wiejski z pomidorami i pieczywem', 250, 'śniadanie', True, False, 'Twarożek z dodatkiem pomidorów i pieczywa.', 'mleko'),
    ('J25', 'wrap śniadaniowy z serkiem śmietanowym i łososiem', 400, 'śniadanie', False, False, 'Zawijany placek z kremowym serkiem i wędzonym łososiem.', 'gluten, mleko, ryba'),
    ('J26', 'wrap śniadaniowy z kurczakiem', 450, 'śniadanie', False, False, 'Zawijany placek z kurczakiem, warzywami i sosem.', 'gluten'),
    ('J27', 'tost z pastą z awokado', 300, 'śniadanie', True, True, 'Grzanka z pastą z dojrzałego awokado i przyprawami.', 'gluten'),
    ('J28', 'smoothie bowl', 350, 'śniadanie', True, True, 'Miska zmiksowanych owoców z dodatkami: nasionami, granolą, owocami.', 'brak'),
    ('J29', 'chia pudding z mlekiem roślinnym i owocami', 250, 'śniadanie', True, True, 'Pudding na bazie nasion chia z mlekiem roślinnym i dodatkami.', 'orzechy'),
    
    # Obiady (J30-J49)
    ('J30', 'spaghetti bolognese', 600, 'obiad', False, False, 'Makaron z sosem mięsnym na bazie pomidorów.', 'gluten'),
    ('J31', 'lasagne z wołowiną i sosem pomidorowym', 700, 'obiad', False, False, 'Warstwowa zapiekanka z mięsem, makaronem i beszamelem.', 'gluten, mleko'),
    ('J32', 'pierogi ruskie', 500, 'obiad', True, False, 'Klasyczne pierogi z farszem z ziemniaków i sera.', 'gluten, mleko'),
    ('J33', 'pierogi z kapustą i grzybami', 450, 'obiad', True, True, 'Tradycyjne pierogi wigilijne z kapustą i suszonymi grzybami.', 'gluten'),
    ('J34', 'pierogi z mięsem', 550, 'obiad', False, False, 'Ciasto pierogowe wypełnione mięsnym farszem.', 'gluten, mleko'),
    ('J35', 'kotlet schabowy z ziemniakami i mizerią', 800, 'obiad', False, False, 'Tradycyjny polski obiad z panierowanym mięsem.', 'gluten, jaja'),
    ('J36', 'nuggetsy z frytkami', 700, 'obiad', False, False, 'Kawałki kurczaka w chrupiącej panierce z frytkami.', 'gluten, jaja'),
    ('J37', 'burger z wołowiną', 700, 'obiad', False, False, 'Bułka z kotletem wołowym, warzywami i sosem.', 'gluten, mleko, jaja'),
    ('J38', 'burger z falafelem', 600, 'obiad', True, True, 'Wegetariański burger z kotletem z ciecierzycy.', 'gluten, sezam'),
    ('J39', 'wrap z kurczakiem', 450, 'obiad', False, False, 'Placek pszenny z grillowanym kurczakiem i warzywami.', 'gluten'),
    ('J40', 'wrap z falafelem', 500, 'obiad', True, True, 'Placek pszenny z falafelem, warzywami i hummusem.', 'gluten'),
    ('J41', 'placki ziemniaczane ze śmietaną', 600, 'obiad', True, False, 'Smażone placki ziemniaczane podawane ze śmietaną.', 'jaja, mleko, gluten'),
    ('J42', 'tofu curry z mlekiem kokosowym i warzywami', 500, 'obiad', True, True, 'Azjatyckie curry z tofu i aromatycznymi przyprawami.', 'soja'),
    ('J43', 'pieczona ciecierzyca z batatami i sosem tahini', 550, 'obiad', True, True, 'Zdrowe danie roślinne z piekarnika.', 'sezam'),
    ('J44', 'lasagne ze szpinakiem i rukolą', 500, 'obiad', True, False, 'Warstwowa lasagne z serem, szpinakiem i rukolą.', 'gluten, mleko'),
    ('J45', 'pulpeciki drobiowe w sosie koperkowym', 600, 'obiad', False, False, 'Delikatne pulpeciki w kremowym sosie.', 'gluten, jaja, mleko'),
    ('J46', 'pad thai z kurczakiem', 700, 'obiad', False, False, 'Tajski makaron ryżowy z kurczakiem, jajkiem i orzeszkami.', 'jaja, orzechy, soja'),
    ('J47', 'pad thai z warzywami', 600, 'obiad', True, True, 'Wersja warzywna tajskiego dania z makaronem.', 'jaja, orzechy, soja'),
    ('J48', 'stir-fry z kurczakiem', 500, 'obiad', False, False, 'Szybkie danie z kurczakiem i warzywami smażone na woku.', 'soja, gluten'),
    ('J49', 'stir-fry z warzywami', 400, 'obiad', True, True, 'Wegetariańska wersja stir-fry z mieszanką warzyw.', 'soja, gluten'),
    
    # Kolacje (J50-J64) - powtórzenia śniadań
    ('J50', 'owsianka z cynamonem i jabłkiem', 250, 'kolacja', True, True, 'Ciepła owsianka z duszonym jabłkiem i przyprawami.', 'gluten'),
    ('J51', 'omlet z warzywami i serem', 300, 'kolacja', True, False, 'Jajeczny placek z dodatkiem warzyw i sera.', 'jaja, mleko'),
    ('J52', 'jajecznica ze szczypiorkiem', 200, 'kolacja', True, False, 'Klasyczna jajecznica z posiekanym szczypiorkiem.', 'jaja'),
    ('J53', 'jajecznica z boczkiem', 300, 'kolacja', False, False, 'Jajecznica z dodatkiem podsmażonego boczku.', 'jaja'),
    ('J54', 'pancakes z kremem orzechowo-czekoladowym', 400, 'kolacja', True, False, 'Amerykańskie naleśniki z kremem typu Nutella.', 'gluten, jaja, mleko, orzechy'),
    ('J55', 'kanapka z szynką, serem i pomidorem', 350, 'kolacja', False, False, 'Klasyczna kanapka na pieczywie z dodatkami.', 'gluten, mleko'),
    ('J56', 'kanapka z pastą jajeczną', 300, 'kolacja', True, False, 'Pieczywo z pastą z jajek, majonezu i przypraw.', 'gluten, jaja'),
    ('J57', 'kanapka z mozarellą', 320, 'kolacja', True, False, 'Kanapka z mozzarellą, pomidorem i bazylią.', 'gluten, mleko'),
    ('J58', 'jogurt naturalny z granolą i owocami', 300, 'kolacja', True, False, 'Lekki posiłek z chrupiącą granolą i świeżymi owocami.', 'mleko, gluten, orzechy'),
    ('J59', 'serek wiejski z pomidorami i pieczywem', 250, 'kolacja', True, False, 'Twarożek z dodatkiem pomidorów i pieczywa.', 'mleko'),
    ('J60', 'wrap śniadaniowy z serkiem śmietanowym i łososiem', 400, 'kolacja', True, False, 'Zawijany placek z kremowym serkiem i wędzonym łososiem.', 'gluten, mleko, ryba'),
    ('J61', 'wrap śniadaniowy z kurczakiem', 450, 'kolacja', False, False, 'Zawijany placek z kurczakiem, warzywami i sosem.', 'gluten'),
    ('J62', 'tost z pastą z awokado', 300, 'kolacja', True, True, 'Grzanka z pastą z dojrzałego awokado i przyprawami.', 'gluten'),
    ('J63', 'smoothie bowl', 350, 'kolacja', True, True, 'Miska zmiksowanych owoców z dodatkami: nasionami, granolą, owocami.', 'brak'),
    ('J64', 'chia pudding z mlekiem roślinnym i owocami', 250, 'kolacja', True, True, 'Pudding na bazie nasion chia z mlekiem roślinnym i dodatkami.', 'orzechy')
]

try:
    # Nawiązanie połączenia
    conn = mysql.connector.connect(**DB_CONFIG)
    cursor = conn.cursor()

    # Optymalizacja - wyłączenie sprawdzania kluczy obcych
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0")
    cursor.execute("SET autocommit = 0")
    cursor.execute("SET unique_checks = 0")

    # Tworzenie tabeli
    cursor.execute("DROP TABLE IF EXISTS jedzenie")
    cursor.execute("""
        CREATE TABLE jedzenie (
            jedzenie_id VARCHAR(10) PRIMARY KEY,
            danie VARCHAR(100),
            liczba_kalorii INT,
            rodzaj VARCHAR(30),
            czy_wegetarianskie BOOLEAN,
            czy_weganskie BOOLEAN,
            opis TEXT,
            alergeny TEXT
        )
    """)

    # Masowe wstawianie danych
    cursor.executemany("""
        INSERT INTO jedzenie (
            jedzenie_id, danie, liczba_kalorii, rodzaj, 
            czy_wegetarianskie, czy_weganskie, opis, alergeny
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, jedzenie_data)

    # Przywrócenie ustawień bazy
    cursor.execute("SET FOREIGN_KEY_CHECKS = 1")
    cursor.execute("SET autocommit = 1")
    cursor.execute("SET unique_checks = 1")
    
    # Zatwierdzenie zmian
    conn.commit()
    print("Tabela 'jedzenie' została pomyślnie utworzona i wypełniona danymi.")

except mysql.connector.Error as err:
    print(f"Błąd: {err}")
    conn.rollback()

finally:
    if conn.is_connected():
        cursor.close()
        conn.close()

Tabela 'jedzenie' została pomyślnie utworzona i wypełniona danymi.


# alergeny, jedzenie_alergen, klient alergen

In [4]:
def kolumna_istnieje(cursor, tabela, kolumna):
    cursor.execute(f"""
        SELECT COUNT(*) FROM INFORMATION_SCHEMA.COLUMNS 
        WHERE table_schema = DATABASE() AND table_name = %s AND column_name = %s
    """, (tabela, kolumna))
    return cursor.fetchone()[0] > 0

try:
    conn = mysql.connector.connect(**DB_CONFIG)
    cursor = conn.cursor()

    # Wyłączenie sprawdzania ograniczeń dla szybszego działania
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0")
    cursor.execute("SET autocommit = 0")
    cursor.execute("SET unique_checks = 0")

    # Tworzenie tabeli alergenów
    cursor.execute("DROP TABLE IF EXISTS alergeny")
    cursor.execute("""
        CREATE TABLE alergeny (
            alergen_id INT PRIMARY KEY AUTO_INCREMENT,
            nazwa VARCHAR(50) UNIQUE
        )
    """)
    cursor.execute("""
        INSERT IGNORE INTO alergeny (nazwa) VALUES
        ('gluten'), ('mleko'), ('jaja'), ('soja'),
        ('orzechy'), ('orzechy ziemne'), ('sezam'),
        ('żelatyna'), ('ryba')
    """)

    # Tabela powiązania jedzenia z alergenami
    cursor.execute("DROP TABLE IF EXISTS jedzenie_alergen")
    cursor.execute("""
        CREATE TABLE jedzenie_alergen (
            jedzenie_id VARCHAR(10),
            alergen_id INT,
            PRIMARY KEY (jedzenie_id, alergen_id),
            FOREIGN KEY (jedzenie_id) REFERENCES jedzenie(jedzenie_id),
            FOREIGN KEY (alergen_id) REFERENCES alergeny(alergen_id)
        )
    """)
    if kolumna_istnieje(cursor, "jedzenie", "alergeny"):
        cursor.execute("ALTER TABLE jedzenie DROP COLUMN alergeny")

    cursor.execute("""
        CREATE TEMPORARY TABLE tmp_jedzenie_alergeny (
            jedzenie_id VARCHAR(10),
            alergen_nazwa VARCHAR(50)
        )
    """)
    cursor.execute("""
        INSERT INTO tmp_jedzenie_alergeny VALUES
        ('J01', 'mleko'), ('J01', 'orzechy'), ('J01', 'soja'), ('J01', 'gluten'),
        ('J02', 'gluten'), ('J03', 'gluten'), ('J04', 'żelatyna'), ('J05', 'mleko'),
        ('J05', 'soja'), ('J05', 'orzechy'), ('J06', 'mleko'), ('J06', 'soja'),
        ('J06', 'orzechy'), ('J07', 'mleko'), ('J07', 'jaja'), ('J07', 'gluten'),
        ('J08', 'gluten'), ('J08', 'sezam'), ('J09', 'orzechy ziemne'), ('J10', 'gluten'),
        ('J10', 'sezam'), ('J12', 'gluten'), ('J12', 'mleko'), ('J12', 'jaja'),
        ('J12', 'soja'), ('J13', 'mleko'), ('J14', 'gluten'), ('J14', 'mleko'),
        ('J14', 'jaja'), ('J14', 'soja'), ('J15', 'gluten'), ('J16', 'jaja'),
        ('J16', 'mleko'), ('J17', 'jaja'), ('J18', 'jaja'), ('J19', 'gluten'),
        ('J19', 'jaja'), ('J19', 'mleko'), ('J19', 'orzechy'), ('J20', 'gluten'),
        ('J20', 'mleko'), ('J21', 'gluten'), ('J21', 'jaja'), ('J22', 'gluten'),
        ('J22', 'mleko'), ('J23', 'mleko'), ('J23', 'gluten'), ('J23', 'orzechy'),
        ('J24', 'mleko'), ('J25', 'gluten'), ('J25', 'mleko'), ('J25', 'ryba'),
        ('J26', 'gluten'), ('J27', 'gluten'), ('J29', 'orzechy'), ('J30', 'gluten'),
        ('J31', 'gluten'), ('J31', 'mleko'), ('J32', 'gluten'), ('J32', 'mleko'),
        ('J33', 'gluten'), ('J34', 'gluten'), ('J34', 'mleko'), ('J35', 'gluten'),
        ('J35', 'jaja'), ('J36', 'gluten'), ('J36', 'jaja'), ('J37', 'gluten'),
        ('J37', 'mleko'), ('J37', 'jaja'), ('J38', 'gluten'), ('J38', 'sezam'),
        ('J39', 'gluten'), ('J40', 'gluten'), ('J41', 'jaja'), ('J41', 'mleko'),
        ('J41', 'gluten'), ('J42', 'soja'), ('J43', 'sezam'), ('J44', 'gluten'),
        ('J44', 'mleko'), ('J45', 'gluten'), ('J45', 'jaja'), ('J45', 'mleko'),
        ('J46', 'jaja'), ('J46', 'orzechy'), ('J46', 'soja'), ('J47', 'jaja'),
        ('J47', 'orzechy'), ('J47', 'soja'), ('J48', 'soja'), ('J48', 'gluten'),
        ('J49', 'soja'), ('J49', 'gluten'), ('J50', 'gluten'), ('J51', 'jaja'),
        ('J51', 'mleko'), ('J52', 'jaja'), ('J53', 'jaja'), ('J54', 'gluten'),
        ('J54', 'jaja'), ('J54', 'mleko'), ('J54', 'orzechy'), ('J55', 'gluten'),
        ('J55', 'mleko'), ('J56', 'gluten'), ('J56', 'jaja'), ('J57', 'gluten'),
        ('J57', 'mleko'), ('J58', 'mleko'), ('J58', 'gluten'), ('J58', 'orzechy'),
        ('J59', 'mleko'), ('J60', 'gluten'), ('J60', 'mleko'), ('J60', 'ryba'),
        ('J61', 'gluten'), ('J62', 'gluten'), ('J64', 'orzechy')
    """)

    cursor.execute("TRUNCATE TABLE jedzenie_alergen")
    cursor.execute("""
        INSERT INTO jedzenie_alergen (jedzenie_id, alergen_id)
        SELECT t.jedzenie_id, a.alergen_id
        FROM tmp_jedzenie_alergeny t
        JOIN alergeny a ON a.nazwa = t.alergen_nazwa
    """)
    cursor.execute("DROP TEMPORARY TABLE IF EXISTS tmp_jedzenie_alergeny")

except mysql.connector.Error as err:
    print(f"Błąd: {err}")
    conn.rollback()

finally:
    if conn.is_connected():
        try:
            while cursor.nextset():
                pass
        except:
            pass
        cursor.close()
        conn.close()  

In [5]:
conn = mysql.connector.connect(**DB_CONFIG)
cursor = conn.cursor(dictionary=True)


    # Wyłączenie sprawdzania ograniczeń dla szybszego działania
cursor.execute("SET FOREIGN_KEY_CHECKS = 0")
cursor.execute("SET autocommit = 0")
cursor.execute("SET unique_checks = 0")

# Tworzenie tabeli alergenów
cursor.execute("DROP TABLE IF EXISTS klient_alergen")
cursor.execute("""
    CREATE TABLE klient_alergen (
    
        klient_id INT,
        alergen_id INT,
        PRIMARY KEY (klient_id, alergen_id),
        FOREIGN KEY (klient_id) REFERENCES klienci(klient_id),
        FOREIGN KEY (alergen_id) REFERENCES alergeny(alergen_id)
    )
""")


# Pobieramy alergeny do słownika: nazwa -> id
cursor.execute("SELECT alergen_id, nazwa FROM alergeny")
alergeny_map = {row['nazwa'].strip().lower(): row['alergen_id'] for row in cursor.fetchall()}

# Pobieramy klientów i ich alergeny
cursor.execute("SELECT klient_id, alergeny FROM klienci")
klienci = cursor.fetchall()

# Przygotowanie danych do wstawienia
klient_alergen_rows = []

for klient in klienci:
    klient_id = klient['klient_id']
    alergeny_raw = klient['alergeny']

    if alergeny_raw:
        alergeny_lista = [a.strip().lower() for a in alergeny_raw.split(',')]
        for nazwa in alergeny_lista:
            alergen_id = alergeny_map.get(nazwa)
            if alergen_id:
                klient_alergen_rows.append((klient_id, alergen_id))

# Wstawienie danych do tabeli klient_alergen
cursor.executemany(
    "INSERT IGNORE INTO klient_alergen (klient_id, alergen_id) VALUES (%s, %s)",
    klient_alergen_rows
)

conn.commit()
cursor.close()
conn.close()